In [1]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy
from IPython.display import display
import re

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from modules.fixture_difficulty_matrix import FixtureDifficultyMatrix
from modules.utils import getDataFilesSorted

import config

In [2]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
scores_filename = f"./results/{CURRENT_DATE}/scores_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 1
TOTAL_BUDGET = 1000

In [3]:
BB_THRESHOLD = 0.35

In [4]:
current_team_names = {"\u0110or\u0111e Petrovi\u0107",
                        "Jordan Pickford",
                        
                        "Gabriel dos Santos Magalhães",
                        "Marcos Senesi Bar\u00f3n",
                        "Trevoh Chalobah",
                        "Jurriën Timber",
                        "James Tarkowski",
                        
                        "Declan Rice",
                        "Jaidon Anthony",
                        "Enzo Fernández",
                        "James Garner",
                        "Harry Wilson",

                        "Igor Thiago Nascimento Rodrigues",
                        "Jean-Philippe Mateta",
                        "Erling Haaland",
        }

In [5]:
# TODO: Change how files are loaded

with open(teams_filename, "r") as f:
    tempJson: list[dict] = json.load(f)

actualJson = tempJson[SELECTED_MODEL_INDEX]

In [6]:
all_player_data = pd.DataFrame.from_records(actualJson["players"])

In [7]:
print(all_player_data)

     clean_sheets  cost  form  gameweek  home_game  ict_index   id  \
0            13.0    59   5.0        26       True       39.4    1   
1             0.0    41   0.0        26       True        0.0    2   
2             0.0    40   0.0        26       True        0.0    3   
3             0.0    39   0.0        26       True        0.0    4   
4            12.0    71   5.8        26       True       79.1    5   
..            ...   ...   ...       ...        ...        ...  ...   
812           0.0    50   1.0        26      False        1.7  813   
813           0.0    45   1.0        26      False        0.4  814   
814           1.0    45   6.0        26      False        2.6  815   
815           0.0    50   2.0        26       True        4.2  816   
816           0.0    55   4.0        26       True        3.2  817   

                             name opposing_team  play_chance  play_percent  \
0               David Raya Martín       BRE,WOL        100.0      0.961538   
1  

In [8]:
currentTeamPlayers = all_player_data.loc[all_player_data["name"].isin(current_team_names)]


for player in current_team_names:
    if(player not in currentTeamPlayers["name"].values):
        raise ValueError(f"player '{player}' not found")

In [9]:
current_team = Team.fromDataFrame(currentTeamPlayers)

In [10]:
current_team_cost = current_team.getTotalCost()
current_team_cost

1000

In [11]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)

all_data = all_data[SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame.from_records(all_data["team"])

In [12]:
selected_team = Team.fromDataFrame(selected_team_df)

In [13]:
display(selected_team)

ID,Name,Cost,ICT Index,Total Points,Form,Normalised Average Fixture Difficulty,Exponential DifficultyCurrent Fixture Difficulty,Position,Availability,Team,Captain,Vice Captain,Score
287,Jordan Pickford,56.00,60.6,107,4.2,0.000,0.000,0.0000,GKP,True,EVE,False,False106.13
670,Robin Roefs,50.00,69.1,105,2.5,0.000,0.000,0.0000,GKP,True,SUN,False,False102.72
260,Marc Guéhi,52.00,101.9,117,3.0,0.000,0.000,0.0000,DEF,True,MCI,False,False122.83
226,Trevoh Chalobah,58.00,83.2,121,4.8,0.000,0.000,0.0000,DEF,True,CHE,False,False125.90
8,Jurriën Timber,64.00,104.0,121,4.2,0.000,0.000,0.0000,DEF,True,ARS,False,False129.21
5,Gabriel dos Santos Magalhães,71.00,79.1,135,5.8,0.000,0.000,0.0000,DEF,True,ARS,True,False139.54
291,James Tarkowski,58.00,100.3,116,4.0,0.000,0.000,0.0000,DEF,True,EVE,False,False121.22
249,João Pedro Junqueira de Jesus,76.00,145.0,122,9.5,0.000,0.000,0.0000,FWD,True,CHE,False,False125.17
136,Igor Thiago Nascimento Rodrigues,70.00,175.5,131,2.8,0.000,0.000,0.0000,FWD,True,BRE,False,True137.84
624,Jarrod Bowen,77.00,148.9,115,6.2,0.000,0.000,0.0000,FWD,True,WHU,False,False122.98


In [14]:
new_players = selected_team - current_team
print(new_players)


Total Score: 719.1845288507103

Goalkeepers:
- Robin Roefs	Score: 102.72	Cost: 50	Fixture Difficulty: 0.0
Defenders:
- Marc Guéhi	Score: 122.83	Cost: 52	Fixture Difficulty: 0.0
Attackers:
- João Pedro Junqueira de Jesus (Captain) 	Score: 125.17	Cost: 76	Fixture Difficulty: 0.0
- Jarrod Bowen	Score: 122.98	Cost: 77	Fixture Difficulty: 0.0
Midfielders:
- Morgan Rogers	Score: 122.31	Cost: 76	Fixture Difficulty: 0.0
- Bruno Guimarães Rodriguez Moura (Vice Captain) 	Score: 123.18	Cost: 70	Fixture Difficulty: 0.0


In [15]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

surplus = current_team_cost - new_team_cost


Added cost: -18


In [16]:
#HEURISTIC = "combined"
#MODE = SolverMode.CHEAPEST_FIRST

In [17]:
# all_player_data["score"] = all_player_data[HEURISTIC] * all_player_data["form"]
deviation = np.std(all_player_data["score"])
scale_factor = deviation
scale_factor, deviation

(35.225822302110515, 35.225822302110515)

In [18]:
matrix = FixtureDifficultyMatrix()
matrix.precomputeFixtureDifficulty(0, config.CURRENT_GAMEWEEK+1, 5, config.CURRENT_SEASON, 1.0)

new_players.recalculateFixtureDifficulty(matrix)
current_team.recalculateFixtureDifficulty(matrix)
# new_players.calculateScore(HEURISTIC)

# selected_team.recalculateFixtureDifficulty(matrix)
# selected_team.calculateScore(HEURISTIC)

# current_team.recalculateFixtureDifficulty(matrix)
# current_team.calculateScore(HEURISTIC)

Team rankings:
{'UNK': 0.047619047619047616, 'ARS': 0.09523809523809523, 'BRE': 0.14285714285714285, 'MUN': 0.19047619047619047, 'EVE': 0.23809523809523808, 'LIV': 0.2857142857142857, 'BOU': 0.3333333333333333, 'NEW': 0.38095238095238093, 'CHE': 0.42857142857142855, 'LEE': 0.47619047619047616, 'SUN': 0.5238095238095238, 'NFO': 0.5714285714285714, 'MCI': 0.6190476190476191, 'BHA': 0.6666666666666666, 'AVL': 0.7142857142857143, 'WOL': 0.7619047619047619, 'TOT': 0.8095238095238095, 'WHU': 0.8571428571428571, 'FUL': 0.9047619047619048, 'CRY': 0.9523809523809523, 'BUR': 1.0, 'LUT': 1.0, 'SHU': 1.0, 'SOU': 1.0, 'IPS': 1.0, 'LEI': 1.0}


In [19]:
print("Current Team:")
display(current_team)
print()
print("Selected Team:")
display(selected_team)

Current Team:


ID,Name,Cost,ICT Index,Total Points,Form,Normalised Average Fixture Difficulty,Exponential DifficultyCurrent Fixture Difficulty,Position,Availability,Team,Captain,Vice Captain,Score
67,Đorđe Petrović,45.00,54.7,73,4.0,0.548,0.575,0.5476,GKP,True,BOU,False,False31.72
287,Jordan Pickford,56.00,60.6,107,4.2,0.452,0.425,0.4524,GKP,True,EVE,False,False60.97
5,Gabriel dos Santos Magalhães,71.00,79.1,135,5.8,0.417,0.371,-0.0818,DEF,True,ARS,False,True87.83
8,Jurriën Timber,64.00,104.0,121,4.2,0.417,0.371,-0.0818,DEF,True,ARS,False,False81.33
72,Marcos Senesi Barón,48.00,121.4,104,4.0,0.548,0.575,0.5476,DEF,True,BOU,False,False46.63
226,Trevoh Chalobah,58.00,83.2,121,4.8,0.476,0.463,0.4762,DEF,True,CHE,False,False67.66
291,James Tarkowski,58.00,100.3,116,4.0,0.452,0.425,0.4524,DEF,True,EVE,False,False69.65
136,Igor Thiago Nascimento Rodrigues,70.00,175.5,131,2.8,0.524,0.537,0.5238,FWD,True,BRE,False,False63.77
283,Jean-Philippe Mateta,75.00,119.4,84,1.0,0.476,0.463,0.4762,FWD,False,CRY,False,False0.00
430,Erling Haaland,149.00,226.1,182,4.8,0.357,0.283,0.3571,FWD,True,MCI,True,False133.29



Selected Team:


ID,Name,Cost,ICT Index,Total Points,Form,Normalised Average Fixture Difficulty,Exponential DifficultyCurrent Fixture Difficulty,Position,Availability,Team,Captain,Vice Captain,Score
287,Jordan Pickford,56.00,60.6,107,4.2,0.000,0.000,0.0000,GKP,True,EVE,False,False106.13
670,Robin Roefs,50.00,69.1,105,2.5,0.619,0.683,0.6190,GKP,True,SUN,False,False32.59
260,Marc Guéhi,52.00,101.9,117,3.0,0.357,0.283,0.3571,DEF,True,MCI,False,False88.06
226,Trevoh Chalobah,58.00,83.2,121,4.8,0.000,0.000,0.0000,DEF,True,CHE,False,False125.90
8,Jurriën Timber,64.00,104.0,121,4.2,0.000,0.000,0.0000,DEF,True,ARS,False,False129.21
5,Gabriel dos Santos Magalhães,71.00,79.1,135,5.8,0.000,0.000,0.0000,DEF,True,ARS,True,False139.54
291,James Tarkowski,58.00,100.3,116,4.0,0.000,0.000,0.0000,DEF,True,EVE,False,False121.22
249,João Pedro Junqueira de Jesus,76.00,145.0,122,9.5,0.476,0.463,0.4762,FWD,True,CHE,True,False67.26
136,Igor Thiago Nascimento Rodrigues,70.00,175.5,131,2.8,0.000,0.000,0.0000,FWD,True,BRE,False,True137.84
624,Jarrod Bowen,77.00,148.9,115,6.2,0.833,0.933,0.8333,FWD,True,WHU,False,False8.24


In [20]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: 558.5379292825717


In [21]:
def countTeams(pPlayers: Team) -> dict[str, int]:
    result: dict[str, int] = dict()
    for player in pPlayers.getPlayers():
        team = player.getTeam()
        currentNum = result.get(team, 0)
        result[team] = currentNum + 1
    return result

In [22]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    # TODO: Store Team players in sorted tree instead of list
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)
    #maxCost = TOTAL_BUDGET
    teamCounts = countTeams(pCurrentTeam)

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        oldPlayerTeam = oldPlayer.getTeam()
        for j in range(len(newPlayersPositionData)):
            teamCountsCopy = deepcopy(teamCounts)

            currentCount = teamCountsCopy.get(oldPlayerTeam, 0)
            teamCountsCopy[oldPlayerTeam] = currentCount - 1
            
            newPlayer = newPlayersPositionData[j]
            newPlayerTeam = newPlayer.getTeam()
            currentCount = teamCountsCopy.get(newPlayerTeam, 0)
            teamCountsCopy[newPlayerTeam] =currentCount + 1
            if (teamCountsCopy[newPlayerTeam] <= 3):
                newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

                if (newCost <= maxCost):
                    allTransfers.append(Transfer(oldPlayer, newPlayer))

    if (len(allTransfers) >= 1):
        bestTransfer = max(allTransfers)
        if(bestTransfer.getScoreDif() > 0):
            return bestTransfer
    else:
        return None

In [23]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [24]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [25]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)
    actualBestTransfer = max(bestTransfers) if len(bestTransfers) > 0 else None
    return actualBestTransfer

In [26]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team, pMatrix: FixtureDifficultyMatrix):

    TRIPLE_CAPT_THRESHOLD = 0.1

    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    if (transferData is None):
        print("No transfer recommended.")
        return pCurrentTeam
    else:
        transferData.updateFixtureDifficulties(pMatrix)
        print("Best transfer:")
        print(transferData)
        oldPlayer = transferData.getOldPlayer()
        newPlayer = transferData.getNewPlayer()
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    

    newTeam.recalculateFixtureDifficulty(pMatrix)

    tripleCaptainCandidates = []
    for player in newTeam.getPlayers():
        if(player.getCurrentDifficulty() < TRIPLE_CAPT_THRESHOLD):
            tripleCaptainCandidates.append(player)
    triplCaptainsSorted = sorted(tripleCaptainCandidates, key=lambda x: (-x.getCurrentDifficulty(), x.getScore()))
    triplCaptain = triplCaptainsSorted[-1]
    print(f"Suggested triple captain: {triplCaptain.getName()}")
    return newTeam

In [27]:
def calcDecay() -> float:
    halfNumGameweeks: int = config.MAX_GAMEWEEKS // 2
    if (config.CURRENT_GAMEWEEK >= halfNumGameweeks):
        remainingWeeks = config.MAX_GAMEWEEKS - config.CURRENT_GAMEWEEK
    else:
        remainingWeeks = halfNumGameweeks - config.CURRENT_GAMEWEEK
    percentLeft: float = remainingWeeks / halfNumGameweeks
    actualThreshold = ((1.0 - percentLeft) * config.MAX_CHIP_DECAY)
    return actualThreshold
    ...

In [28]:
def averageDifficulty(pPlayers: list[Player]) -> float:
    totalFixtureDifficulty: float = 0.0
    for player in pPlayers:
        totalFixtureDifficulty += player.getCurrentDifficulty()
    avgDifficulty: float = totalFixtureDifficulty / len(pPlayers)
    return avgDifficulty

In [29]:
def checkBenchBoost(pTeam: BenchTeam):

    avgDifficulty: float = averageDifficulty(pTeam.getBenchPlayerList())
    
    actualThreshold = calcDecay() + BB_THRESHOLD

    if (avgDifficulty <= actualThreshold):
        toPrint = "Bench boost is suggested. Threshold: {:.3f} Average Fixture Difficulty: {:.3f}".format(actualThreshold, avgDifficulty)
        display(toPrint)
    else:
        toPrint = "Bench boost NOT needed. Threshold: {:.3f} Average Fixture Difficulty: {:.3f}".format(actualThreshold, avgDifficulty)
        display(toPrint)

In [30]:
def checkFreeHit(pOldTeam: BenchTeam, pNewTeam: BenchTeam):
    oldTeamPlayers = set(x.getName() for x in pOldTeam.getAllPlayerList())
    newTeamPlayers = set(x.getName() for x in pNewTeam.getAllPlayerList())
    uniqueNewPlayers = newTeamPlayers - oldTeamPlayers
    diffPercent = len(uniqueNewPlayers) / len(newTeamPlayers)
    
    THRESHOLD = 0.9
    actualThreshold = THRESHOLD - calcDecay()

    avgDifficulty: float = averageDifficulty(pNewTeam.getAllPlayerList())
    FREE_HIT_DIFF_THRESHOLD = 0.45
    stats = "Threshold: {:.3f} Team difference: {:.3f} Difficulty: {:.3f}".format(actualThreshold, diffPercent, avgDifficulty)

    if ((diffPercent >= actualThreshold) and (avgDifficulty <= FREE_HIT_DIFF_THRESHOLD)):
        display(f"Free Hit recommended. {stats}")
    else:
        display(f"Free Hit NOT needed. {stats}")

In [31]:
newTeam = getNewTeam(current_team, selected_team, new_players, matrix)
newTeam = newTeam.toBenchTeam()
display(newTeam)
display(f"New total score: {newTeam.getTotalScore()}")
display(f"New total cost: {newTeam.getTotalCost()}")
checkBenchBoost(newTeam)
checkFreeHit(newTeam, selected_team.toBenchTeam())

Best transfer:
Transfer from Trevoh Chalobah -> Marc Guéhi:
Old player: Trevoh Chalobah	Score: 67.66	Cost: 58	Fixture Difficulty: 0.47619047619047616
New player: Marc Guéhi	Score: 88.06	Cost: 52	Fixture Difficulty: 0.35714285714285715
- Cost change: -6
- Score change: 20.41
Suggested triple captain: Gabriel dos Santos Magalhães


ID,Name,Cost,ICT Index,Total Points,Form,Normalised Average Fixture Difficulty,Exponential DifficultyCurrent Fixture Difficulty,Position,Availability,Team,Captain,Vice Captain,Score
287,Jordan Pickford,56.00,60.6,107,4.2,0.452,0.425,0.4524,GKP,True,EVE,False,False60.97
260,Marc Guéhi,52.00,101.9,117,3.0,0.357,0.283,0.3571,DEF,True,MCI,False,True88.06
5,Gabriel dos Santos Magalhães,71.00,79.1,135,5.8,0.417,0.371,-0.0818,DEF,True,ARS,False,False87.83
8,Jurriën Timber,64.00,104.0,121,4.2,0.417,0.371,-0.0818,DEF,True,ARS,False,False81.33
291,James Tarkowski,58.00,100.3,116,4.0,0.452,0.425,0.4524,DEF,True,EVE,False,False69.65
430,Erling Haaland,149.00,226.1,182,4.8,0.357,0.283,0.3571,FWD,True,MCI,True,False133.29
136,Igor Thiago Nascimento Rodrigues,70.00,175.5,131,2.8,0.524,0.537,0.5238,FWD,True,BRE,False,False63.77
21,Declan Rice,75.00,164.5,134,3.0,0.417,0.371,-0.0818,MID,True,ARS,False,False81.82
237,Enzo Fernández,69.00,180.5,116,6.0,0.476,0.463,0.4762,MID,True,CHE,False,False66.88
303,James Garner,52.00,130.8,109,3.5,0.452,0.425,0.4524,MID,True,EVE,False,False61.37


'New total score: 938.4920231685111'

'New total cost: 994'

'Bench boost NOT needed. Threshold: 0.491 Average Fixture Difficulty: 0.554'

'Free Hit NOT needed. Threshold: 0.759 Team difference: 0.400 Difficulty: 0.286'